<a href="https://colab.research.google.com/github/NvdSuni/Thesis-code-complete/blob/main/Combined_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt

import numpy as np
from PIL import Image
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
import joblib

In [ ]:
!pip install vit-keras
!pip install timm

In [ ]:
import google
import tensorflow as tf
import matplotlib
import numpy as np
import PIL
import vit_keras
import timm
import joblib
from sklearn import __version__ as sklearn_version

print(f"scikit-learn: {sklearn_version}")

print(f"google.colab: {google.colab.__version__}")
print(f"tensorflow: {tf.__version__}")
print(f"matplotlib: {matplotlib.__version__}")
print(f"numpy: {np.__version__}")
print(f"PIL: {PIL.__version__}")
print(f"vit-keras: {vit_keras.__version__}")
print(f"timm: {timm.__version__}")
print(f"joblib: {joblib.__version__}")

In [ ]:
X_train_Xray1 = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/cnn_features_X_train_Xray1.npy")
X_train_Xray2 = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/cnn_features_X_train_Xray2.npy")
X_val_Xray = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/cnn_features_X_val_Xray.npy")
X_train_MRNet = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/cnn_features_X_train_MRNet2.npy")
X_val_MRNet = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/cnn_features_X_val_MRNet2.npy")
X_test_MRNet = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/cnn_features_X_test_MRNet.npy")
X_test_Xray = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/cnn_features_X_test_Xray.npy")

train_labels_Xray = np.load('/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/train_labels_complete_Xray.npy')
val_labels_Xray = np.load('/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/val_labels_complete_Xray.npy')
train_labels_MRNet = np.load('/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/y_train_MRNet.npy')
val_labels_MRNet = np.load('/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/y_val_MRNet.npy')

X_test_MRNet = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/X_test_MRNet_reduced.npy")
X_test_Xray = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/X_test_Xray_reduced.npy")
y_test_MRNet = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/y_test_MRNet.npy")
y_test_Xray = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/test_labels_complete_Xray.npy")

#PCA APPLICATION

In [ ]:
from sklearn.decomposition import PCA

#Define the desired reduced dimension
reduced_dimension = 64

def reshape_data(data):
    return data.reshape(data.shape[0], -1)

X_train_MRNet_2D = reshape_data(X_train_MRNet2)
X_test_MRNet_2D = reshape_data(X_test_MRNet)
X_val_MRNet_2D = reshape_data(X_val_MRNet2)
X_train_Xray_2D = reshape_data(X_train_Xray1)
X_test_Xray_2D = reshape_data(X_test_Xray)
X_val_Xray_2D = reshape_data(X_val_Xray)

#Apply PCA for dimensionality reduction
def apply_pca(data, reduced_dimension):
    pca = PCA(n_components=reduced_dimension)
    reduced_data = pca.fit_transform(data)
    return pca, reduced_data

pca_MRNet, X_train_MRNet_reduced = apply_pca(X_train_MRNet_2D, reduced_dimension)
X_train_MRNet_reduced = pca_MRNet.transform(X_train_MRNet_2D)
X_val_MRNet_reduced = pca_MRNet.transform(X_val_MRNet_2D)
X_test_MRNet_reduced = pca_MRNet.transform(X_test_MRNet_2D)

pca_Xray, X_train_Xray_reduced1 = apply_pca(X_train_Xray_2D, reduced_dimension)
X_train_Xray_reduced = pca_MRNet.transform(X_train_Xray_2D)
X_val_Xray_reduced = pca_Xray.transform(X_val_Xray_2D)
X_test_Xray_reduced = pca_Xray.transform(X_test_Xray_2D)


In [ ]:
np.save("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/X_train_MRNet_reduced.npy", X_train_MRNet_reduced)
np.save("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/X_train_Xray_reduced1.npy", X_train_Xray_reduced1)
np.save("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/X_val_MRNet_reduced.npy", X_val_MRNet_reduced)
np.save("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/X_val_Xray_reduced.npy", X_val_Xray_reduced)
np.save("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/X_test_MRNet_reduced.npy", X_test_MRNet_reduced)
np.save("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/X_test_Xray_reduced.npy", X_test_Xray_reduced)

#CONCATENATING GROUND-TRUTH LABELS

In [ ]:
X_train_combined_inputs = np.concatenate((X_train_MRNet_reduced, X_train_Xray_reduced), axis = 0)
X_val_combined_inputs = np.concatenate((X_val_MRNet_reduced, X_val_Xray_reduced), axis = 0)

In [ ]:
train_labels_MRNet_binary = train_labels_MRNet_binary.reshape((-1, 1))
print(train_labels_MRNet_binary.shape, train_labels_Xray.shape)

In [ ]:
#Define a mapping from multi-class labels to grade
label_to_grade = {
    (1, 0, 0, 0, 0): 0,
    (0, 1, 0, 0, 0): 1,
    (0, 0, 1, 0, 0): 2,
    (0, 0, 0, 1, 0): 3,
    (0, 0, 0, 0, 1): 4
}

y_train_Xray = []

for row in train_labels_Xray:
    grade = label_to_grade[tuple(row)]
    y_train_Xray.append(grade)


y_train_Xray = np.array(y_train_Xray)

In [ ]:
y_val_Xray = []

for row in val_labels_Xray:
    grade = label_to_grade[tuple(row)]
    y_val_Xray.append(grade)


y_val_Xray = np.array(y_val_Xray)

In [ ]:
#Adjust train_labels_MRNet_binary
train_labels_MRNet_adjusted = train_labels_MRNet + 5
val_labels_MRNet_adjusted = val_labels_MRNet + 5

In [ ]:
#Concatenate the labels
y_train_combined = np.concatenate((train_labels_MRNet_adjusted, y_train_Xray), axis=0)
y_val_combined = np.concatenate((val_labels_MRNet_adjusted, y_val_Xray ), axis=0)

In [ ]:
from sklearn.utils import shuffle
X_train_combined, y_train_combined = shuffle(X_train_combined_inputs, y_train_combined, random_state = 42)
X_val_combined, y_val_combined = shuffle(X_val_combined_inputs, y_val_combined, random_state = 42)

In [ ]:
np.save("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/X_train_combined.npy", X_train_combined)
np.save("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/X_val_combined.npy", X_val_combined)
np.save("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/y_train_combined.npy", y_train_combined)
np.save("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/y_val_combined.npy", y_val_combined)

In [ ]:
X_test_MRNet = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/X_test_MRNet_reduced.npy")
X_test_Xray = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/X_test_Xray_reduced.npy")
y_test_MRNet = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/y_test_MRNet.npy")
y_test_Xray = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/test_labels_complete_Xray.npy")

In [ ]:
X_test_combined_inputs = np.concatenate((X_test_MRNet, X_test_Xray), axis = 0)
print(X_test_combined_inputs.shape)

In [ ]:
y_test_Xray_adjusted = np.argmax(y_test_Xray, axis=1)
y_test_MRNet_adjusted = y_test_MRNet + 5
y_test_combined = np.concatenate((y_test_MRNet_adjusted, y_test_Xray_adjusted), axis=0)
X_test_combined, y_test_combined = shuffle(X_test_combined_inputs, y_test_combined, random_state = 42)

In [ ]:
np.save("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/X_test_combined.npy", X_test_combined)
np.save("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/y_test_combined.npy", y_test_combined)